# AI策略比赛--参赛作品-- 作者：woshisilvio
### meetup综合案例运用 采用 大盘相对收益率因子+综合大盘风控止盈止损 模拟中证150指数增强
# 训练集：10年-18年
# 测试集：18年-21年10月
# 因子数：3
# 算法：stockraner
# 标注：未来5日收益率
# 止盈止损：60%/5%
# 大盘风控：指数的macd死叉
# 策略逻辑：找到中证150中成长最快的一组股票池交由AI做训练，找出强趋势股，月线金叉以上的开仓做多
# 寻找在牛市中可以做波段增强的一类股票，对其进行排名打分
# 策略改进方向：股票池，因子池，回归分析，pca主成分分析降维，减少损失
# 算法改进方向：训练深度降低，或采用lightgbm
# 强势：牛市中拥有较为稳定的成长风格收益
# 劣势：在熊市和震荡风格中表现不佳
 

In [1]:
# 本代码由可视化策略环境自动生成 2021年12月3日 14:00
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


# 回测引擎：初始化函数，只执行一次
def m63_initialize_bigquant_run(context):
    # 加载预测数据
    context.ranker_prediction = context.options['data'].read_df()

    # 系统已经设置了默认的交易手续费和滑点，要修改手续费可使用如下函数
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    # 预测数据，通过options传入进来，使用 read_df 函数，加载到内存 (DataFrame)
    # 设置买入的股票数量，这里买入预测股票列表排名靠前的5只
    stock_count = 5
    # 每只的股票的权重，如下的权重分配会使得靠前的股票分配多一点的资金，[0.339160, 0.213986, 0.169580, ..]
    context.stock_weights = T.norm([1 / math.log(i + 2) for i in range(0, stock_count)])
    # 设置每只股票占用的最大资金比例
    context.max_cash_per_instrument =0.1
    context.options['hold_days'] = 5

    from zipline.finance.slippage import SlippageModel
    class FixedPriceSlippage(SlippageModel):
        def process_order(self, data, order, bar_volume=0, trigger_check_price=0):
            if order.limit is None:
                price_field = self._price_field_buy if order.amount > 0 else self._price_field_sell
                price = data.current(order.asset, price_field)
            else:
                price = data.current(order.asset, self._price_field_buy)
            # 返回希望成交的价格和数量
            return (price, order.amount)
    # 设置price_field,默认是开盘买入，收盘卖出
    context.fix_slippage = FixedPriceSlippage(price_field_buy='open', price_field_sell='close')
    context.set_slippage(us_equities=context.fix_slippage)
# 回测引擎：每日数据处理函数，每天执行一次
def m63_handle_data_bigquant_run(context, data):
    # 获取当前持仓
    positions = {e.symbol: p.amount * p.last_sale_price
                 for e, p in context.portfolio.positions.items()}
    
    today = data.current_dt.strftime('%Y-%m-%d')
    # 按日期过滤得到今日的预测数据
    ranker_prediction = context.ranker_prediction[
        context.ranker_prediction.date == today]
    try:
    #大盘风控模块，读取风控数据    
        benckmark_risk=ranker_prediction['bm_0'].values[0]
        if benckmark_risk > 0:
            for instrument in positions.keys():
                context.order_target(context.symbol(instrument), 0)
                print(today,'大盘风控止损触发,全仓卖出')
                return
    except:
        print('缺失风控数据！')
        
    #当risk为1时，市场有风险，全部平仓，不再执行其它操作
     
    
    # 1. 资金分配
    # 平均持仓时间是hold_days，每日都将买入股票，每日预期使用 1/hold_days 的资金
    # 实际操作中，会存在一定的买入误差，所以在前hold_days天，等量使用资金；之后，尽量使用剩余资金（这里设置最多用等量的1.5倍）
    is_staging = context.trading_day_index < context.options['hold_days'] # 是否在建仓期间（前 hold_days 天）
    cash_avg = context.portfolio.portfolio_value / context.options['hold_days']
    cash_for_buy = min(context.portfolio.cash, (1 if is_staging else 1.5) * cash_avg)
    cash_for_sell = cash_avg - (context.portfolio.cash - cash_for_buy)
   
    
    # 2. 根据需要加入移动止赢止损模块、固定天数卖出模块、ST或退市股卖出模块
    stock_sold = [] # 记录卖出的股票，防止多次卖出出现空单
    
    #------------------------START:止赢止损模块(含建仓期)---------------
    current_stopwin_stock=[]
    current_stoploss_stock = []   
    positions_cost={e.symbol:p.cost_basis for e,p in context.portfolio.positions.items()}
    if len(positions)>0:
        for instrument in positions.keys():
            stock_cost=positions_cost[instrument]  
            stock_market_price=data.current(context.symbol(instrument),'price')  
            volume_since_buy = data.history(context.symbol(instrument), 'volume', 6, '1d')
            # 赚9%且为可交易状态就止盈
            if stock_market_price/stock_cost-1>=0.6 and data.can_trade(context.symbol(instrument)):
                context.order_target_percent(context.symbol(instrument),0)
                cash_for_sell -= positions[instrument]
                current_stopwin_stock.append(instrument)
            # 亏5%并且为可交易状态就止损
            if stock_market_price/stock_cost-1 <= -0.05 and data.can_trade(context.symbol(instrument)):   
                context.order_target_percent(context.symbol(instrument),0)
                cash_for_sell -= positions[instrument]
                current_stoploss_stock.append(instrument)
            # 放天量  止损：
#             if  (volume_since_buy[0]>1.5*volume_since_buy[1]) |(volume_since_buy[0]>1.5*(volume_since_buy[1]+volume_since_buy[2]+volume_since_buy[3]+volume_since_buy[4]+volume_since_buy[5])/5):
#                 context.order_target_percent(context.symbol(instrument),0)
#                 cash_for_sell -= positions[instrument]
#                 current_stoploss_stock.append(instrument)
        if len(current_stopwin_stock)>0:
            print(today,'止盈股票列表',current_stopwin_stock)
            stock_sold += current_stopwin_stock
        if len(current_stoploss_stock)>0:
            print(today,'止损股票列表',current_stoploss_stock)
            stock_sold += current_stoploss_stock
    #--------------------------END: 止赢止损模块--------------------------
    
    #--------------------------START：持有固定天数卖出(不含建仓期)-----------
    current_stopdays_stock = []
    positions_lastdate = {e.symbol:p.last_sale_date for e,p in context.portfolio.positions.items()}
    # 不是建仓期（在前hold_days属于建仓期）
    if not is_staging:
        for instrument in positions.keys():
            #如果上面的止盈止损已经卖出过了，就不要重复卖出以防止产生空单
            if instrument in stock_sold:
                continue
            # 今天和上次交易的时间相隔hold_days就全部卖出 datetime.timedelta(context.options['hold_days'])也可以换成自己需要的天数，比如datetime.timedelta(5)
            if data.current_dt - positions_lastdate[instrument]>=datetime.timedelta(22) and data.can_trade(context.symbol(instrument)):
                context.order_target_percent(context.symbol(instrument), 0)
                current_stopdays_stock.append(instrument)
                cash_for_sell -= positions[instrument]
        if len(current_stopdays_stock)>0:        
            print(today,'固定天数卖出列表',current_stopdays_stock)
            stock_sold += current_stopdays_stock
    #-------------------------  END:持有固定天数卖出-----------------------
    
    #-------------------------- START: ST和退市股卖出 ---------------------  
    st_stock_list = []
    for instrument in positions.keys():
        try:
            instrument_name = ranker_prediction[ranker_prediction.instrument==instrument].name.values[0]
            # 如果股票状态变为了st或者退市 则卖出
            if 'ST' in instrument_name or '退' in instrument_name:
                if instrument in stock_sold:
                    continue
                if data.can_trade(context.symbol(instrument)):
                    context.order_target(context.symbol(instrument), 0)
                    st_stock_list.append(instrument)
                    cash_for_sell -= positions[instrument]
        except:
            continue
    if st_stock_list!=[]:
        print(today,'持仓出现st股/退市股',st_stock_list,'进行卖出处理')    
        stock_sold += st_stock_list

    #-------------------------- END: ST和退市股卖出 --------------------- 
    
    
    # 3. 生成轮仓卖出订单：hold_days天之后才开始卖出；对持仓的股票，按机器学习算法预测的排序末位淘汰
    if not is_staging and cash_for_sell > 0:
        instruments = list(reversed(list(ranker_prediction.instrument[ranker_prediction.instrument.apply(
                lambda x: x in positions)])))
        for instrument in instruments:
            # 如果资金够了就不卖出了
            if cash_for_sell <= 0:
                break
            #防止多个止损条件同时满足，出现多次卖出产生空单
            if instrument in stock_sold:
                continue
            context.order_target(context.symbol(instrument), 0)
            cash_for_sell -= positions[instrument]
            stock_sold.append(instrument)

    # 4. 生成轮仓买入订单：按机器学习算法预测的排序，买入前面的stock_count只股票
    # 计算今日跌停的股票
    dt_list = list(ranker_prediction[ranker_prediction.price_limit_status_0==1].instrument)
    # 计算今日ST/退市的股票
    st_list = list(ranker_prediction[ranker_prediction.name.str.contains('ST')|ranker_prediction.name.str.contains('退')].instrument)
    # 计算所有禁止买入的股票池
    banned_list = stock_sold+dt_list+st_list
    buy_cash_weights = context.stock_weights
    buy_instruments=[k for k in list(ranker_prediction.instrument) if k not in banned_list][:len(buy_cash_weights)]
    max_cash_per_instrument = context.portfolio.portfolio_value * context.max_cash_per_instrument
    for i, instrument in enumerate(buy_instruments):
        cash = cash_for_buy * buy_cash_weights[i]
        if cash > max_cash_per_instrument - positions.get(instrument, 0):
            # 确保股票持仓量不会超过每次股票最大的占用资金量
            cash = max_cash_per_instrument - positions.get(instrument, 0)
        if cash > 0:
            context.order_value(context.symbol(instrument), cash)
    


# 回测引擎：准备数据，只执行一次
def m63_prepare_bigquant_run(context):
    context.status_df = D.features(instruments =context.instruments,start_date = context.start_date, end_date = context.end_date, 
                           fields=['st_status_0','price_limit_status_0','price_limit_status_1'])

def m63_before_trading_start_bigquant_run(context, data):
    # 获取涨跌停状态数据
    df_price_limit_status = context.ranker_prediction.set_index('date')
    today=data.current_dt.strftime('%Y-%m-%d')
    # 得到当前未完成订单
    for orders in get_open_orders().values():
        # 循环，撤销订单
        for _order in orders:
            ins=str(_order.sid.symbol)
            try:
                #判断一下如果当日涨停，则取消卖单
                if  df_price_limit_status[df_price_limit_status.instrument==ins].price_limit_status_0.ix[today]>2 and _order.amount<0:
                    cancel_order(_order)
                    print(today,'尾盘涨停取消卖单',ins) 
            except:
                continue

m1 = M.instruments.v2(
    start_date='2010-01-01',
    end_date='2018-01-01',
    market='CN_STOCK_A',
    instrument_list="""002714.SZA
000620.SZA
600066.SHA
600887.SHA
002415.SZA
300450.SZA
000622.SZA
002690.SZA
300601.SZA
000776.SZA
600519.SHA
000333.SZA
002035.SZA
002236.SZA
300015.SZA
600436.SHA
600566.SHA
600201.SHA
600705.SHA
000156.SZA
002677.SZA
002032.SZA
600681.SHA
603601.SHA
002372.SZA
603019.SHA
300571.SZA
002475.SZA
300033.SZA
600346.SHA
601888.SHA
600763.SHA
603799.SHA
603690.SHA
002044.SZA
300253.SZA
000651.SZA
002085.SZA
600903.SHA
600340.SHA
300296.SZA
002572.SZA
300176.SZA
000963.SZA
000403.SZA
002747.SZA
000789.SZA
002508.SZA
300347.SZA
300401.SZA
300666.SZA
000732.SZA
	
000049.SZA
300451.SZA
603986.SHA
300684.SZA
000681.SZA
000671.SZA
002230.SZA
600867.SHA
300059.SZA
002020.SZA
300457.SZA
300136.SZA
300308.SZA
000661.SZA
002304.SZA
600276.SHA
600563.SHA
601012.SHA
000703.SZA
002410.SZA
002358.SZA
300725.SZA
300226.SZA
300675.SZA
300324.SZA
600801.SHA
000048.SZA
300383.SZA
300285.SZA
300459.SZA
002008.SZA
603288.SHA
002049.SZA
002311.SZA
600305.SHA
300377.SZA
002746.SZA
603638.SHA
002507.SZA
002252.SZA
002271.SZA
300482.SZA
000002.SZA
600612.SHA
002081.SZA
002013.SZA
300357.SZA""",
    max_count=0
)

m2 = M.use_datasource.v1(
    instruments=m1.data,
    datasource_id='bar1d_CN_STOCK_A',
    start_date='',
    end_date=''
)

m3 = M.input_features.v1(
    features="""# #号开始的表示注释
# 多个特征，每行一个，可以包含基础特征和衍生特征


avg_turn_15/turn_0
mf_net_amount_xl_0

alpha4=close_0*avg_turn_0+close_1*avg_turn_1+close_2*avg_turn_2
"""
)

m52 = M.input_features.v1(
    features_ds=m3.data,
    features="""# #号开始的表示注释
# 多个特征，每行一个，可以包含基础特征和衍生特征
#周线金叉
cond1=sum(ta_macd_dif(close_0,2,4,4),5)>sum(ta_macd_dea(close_0,2,4,4),5)
cond2=close_0>mean(close_0, 25)
cond3=sum(ta_macd_dea(close_0,2,4,4),5)>0.2
price_limit_status_0
cond4=st_status_0<1"""
)

m15 = M.general_feature_extractor.v7(
    instruments=m1.data,
    features=m52.data,
    start_date='',
    end_date='',
    before_start_days=90
)

m16 = M.derived_feature_extractor.v3(
    input_data=m15.data,
    features=m52.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False
)

m67 = M.features_short.v1(
    input_1=m3.data
)

m9 = M.instruments.v2(
    start_date=T.live_run_param('trading_date', '2018-01-01'),
    end_date=T.live_run_param('trading_date', '2021-10-30'),
    market='CN_STOCK_A',
    instrument_list="""300782.SZA
605358.SHA
603290.SHA
603392.SHA
601865.SHA
300759.SZA
300750.SZA
300677.SZA
002607.SZA
603259.SHA
300751.SZA
603613.SHA
601100.SHA
300763.SZA
002568.SZA
300724.SZA
603345.SHA
600763.SHA
603713.SHA
300595.SZA
300014.SZA
603712.SHA
300760.SZA
603317.SHA
002791.SZA
601066.SHA
002985.SZA
300661.SZA
300347.SZA
300777.SZA
603129.SHA
300454.SZA
601888.SHA
605111.SHA
603638.SHA
300850.SZA
600809.SHA
002414.SZA
603893.SHA
002967.SZA
600132.SHA
603605.SHA
300015.SZA
603267.SHA
300012.SZA
600882.SHA
300684.SZA
300390.SZA
300769.SZA
300748.SZA
000799.SZA
300767.SZA
300775.SZA
603737.SHA
300601.SZA
601698.SHA
300841.SZA
002975.SZA
603501.SHA
300122.SZA
300677.SZA
603392.SHA
002791.SZA
601865.SHA
300759.SZA
002568.SZA
603613.SHA
300014.SZA
601100.SHA
300763.SZA
300274.SZA
601633.SHA
603501.SHA
002709.SZA
603317.SHA
300661.SZA
002985.SZA
600882.SHA
300598.SZA
300777.SZA
300552.SZA
300346.SZA
002475.SZA
605111.SHA
300850.SZA
300526.SZA
601012.SHA
603893.SHA
002967.SZA
000858.SZA
603267.SHA
000568.SZA
603638.SHA
000708.SZA
603456.SHA
000995.SZA
600399.SHA
300767.SZA
300595.SZA
300347.SZA
600763.SHA
300751.SZA
600316.SHA
300775.SZA
603208.SHA
600862.SHA
002241.SZA
002706.SZA
300390.SZA
601698.SHA
002541.SZA
002607.SZA
000733.SZA
000596.SZA
603345.SHA
300151.SZA
300496.SZA
002705.SZA
002756.SZA
603185.SHA
002850.SZA
000661.SZA
002600.SZA
300724.SZA
600584.SHA
002414.SZA
300223.SZA
002920.SZA
603906.SHA
002714.SZA
600966.SHA
300083.SZA
300601.SZA
600438.SHA
002812.SZA
002459.SZA
603027.SHA
300015.SZA
300763.SZA
002709.SZA
688202.SHA
000422.SZA
300769.SZA
300751.SZA
300343.SZA
605117.SHA
300827.SZA
601633.SHA
603026.SHA
002240.SZA
002326.SZA
002487.SZA
000762.SZA
300432.SZA
603396.SHA
300363.SZA
603985.SHA
000155.SZA
002594.SZA
600399.SHA
600702.SHA
300171.SZA
002176.SZA
000733.SZA
300750.SZA
601127.SHA
002812.SZA
603260.SHA
600610.SHA
601012.SHA
003022.SZA
603127.SHA
000301.SZA
002585.SZA
688198.SHA
002245.SZA
300693.SZA
600096.SHA
300568.SZA
300382.SZA
300443.SZA
003031.SZA
605376.SHA
603806.SHA
603223.SHA
688116.SHA
002529.SZA
600141.SHA
600956.SHA
300035.SZA
300316.SZA
002460.SZA
600110.SHA
300671.SZA
002407.SZA
600532.SHA
688599.SHA
002472.SZA
600499.SHA
600111.SHA
600884.SHA
300696.SZA
603267.SHA""",
    max_count=0
)

m17 = M.general_feature_extractor.v7(
    instruments=m9.data,
    features=m52.data,
    start_date='',
    end_date='',
    before_start_days=90
)

m18 = M.derived_feature_extractor.v3(
    input_data=m17.data,
    features=m52.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False
)

m5 = M.instruments.v2(
    start_date='2010-01-01',
    end_date='2019-01-01',
    market='CN_STOCK_A',
    instrument_list='000300.HIX',
    max_count=0
)

m10 = M.input_features.v1(
    features="""
# #号开始的表示注释，注释需单独一行
# 多个特征，每行一个，可以包含基础特征和衍生特征，特征须为本平台特征
close
instrument

"""
)

m20 = M.use_datasource.v1(
    instruments=m5.data,
    features=m10.data,
    datasource_id='bar1d_index_CN_STOCK_A',
    start_date='',
    end_date='',
    m_cached=False
)

m11 = M.instruments.v2(
    start_date='2010-01-01',
    end_date='2019-01-01',
    market='CN_STOCK_A',
    instrument_list='',
    max_count=0
)

m12 = M.input_features.v1(
    features="""
# #号开始的表示注释，注释需单独一行
# 多个特征，每行一个，可以包含基础特征和衍生特征，特征须为本平台特征
close
open
low
high
adjust_factor"""
)

m29 = M.use_datasource.v1(
    instruments=m11.data,
    features=m12.data,
    datasource_id='bar1d_CN_STOCK_A',
    start_date='',
    end_date=''
)

m21 = M.input_features.v1(
    features="""
# #号开始的表示注释，注释需单独一行
# 多个特征，每行一个，可以包含基础特征和衍生特征，特征须为本平台特征
bmret=close/shift(close,1)-1
"""
)

m22 = M.derived_feature_extractor.v3(
    input_data=m20.data,
    features=m21.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False,
    user_functions={}
)

m23 = M.select_columns.v3(
    input_ds=m22.data,
    columns_ds=m10.data,
    columns='',
    reverse_select=True
)

m26 = M.input_features.v1(
    features="""
# #号开始的表示注释，注释需单独一行
# 多个特征，每行一个，可以包含基础特征和衍生特征，特征须为本平台特征
relative_ret=stockret-bmret
relative_ret_5=sum(relative_ret,5)
relative_ret_30=sum(relative_ret,30)"""
)

m30 = M.input_features.v1(
    features="""
# #号开始的表示注释，注释需单独一行
# 多个特征，每行一个，可以包含基础特征和衍生特征，特征须为本平台特征
stockret=close/shift(close,1)-1"""
)

m28 = M.derived_feature_extractor.v3(
    input_data=m29.data,
    features=m30.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False,
    user_functions={}
)

m27 = M.select_columns.v3(
    input_ds=m28.data,
    columns_ds=m12.data,
    columns='',
    reverse_select=True
)

m24 = M.join.v3(
    data1=m23.data,
    data2=m27.data,
    on='date',
    how='inner',
    sort=False
)

m25 = M.derived_feature_extractor.v3(
    input_data=m24.data,
    features=m26.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False,
    user_functions={}
)

m31 = M.filter.v3(
    input_data=m25.data,
    expr='(relative_ret_5>0)&(relative_ret_30>0)&(rank(relative_ret_30)>0.8)',
    output_left_data=False
)

m32 = M.select_columns.v3(
    input_ds=m31.data,
    columns='date,instrument',
    reverse_select=False
)

m33 = M.join.v3(
    data1=m2.data,
    data2=m32.data,
    on='date,instrument',
    how='inner',
    sort=False
)

m34 = M.auto_labeler_on_datasource.v1(
    input_data=m33.data,
    label_expr="""# #号开始的表示注释
# 0. 每行一个，顺序执行，从第二个开始，可以使用label字段
# 1. 可用数据字段见 https://bigquant.com/docs/develop/datasource/deprecated/history_data.html
# 2. 可用操作符和函数见 `表达式引擎 <https://bigquant.com/docs/develop/bigexpr/usage.html>`_

# 计算收益：5日收盘价(作为卖出价格)除以明日开盘价(作为买入价格)
#shift(close, -5) / shift(open, -1)

# 极值处理：用1%和99%分位的值做clip
#clip(label, all_quantile(label, 0.01), all_quantile(label, 0.99))

# 将分数映射到分类，这里使用20个分类
#all_wbins(label, 20)

# 过滤掉一字涨停的情况 (设置label为NaN，在后续处理和训练中会忽略NaN的label)
#where(shift(high, -1) == shift(low, -1), NaN, label)
# 计算收益：2日开盘价(作为卖出价格)除以明日开盘价(作为买入价格)
(shift(close, -5) / shift(open, -1)-1)

# 极值处理：用1%和99%分位的值做clip
clip(label, all_quantile(label, 0.01), all_quantile(label, 0.99))
all_wbins(label, 20)
# 过滤掉一字涨停的情况 (设置label为NaN，在后续处理和训练中会忽略NaN的label)
where(shift(high, -1) == shift(low, -1), NaN, label)
#where(label>0.5, NaN, label)
#where(label<-0.5, NaN, label)
""",
    drop_na_label=True,
    cast_label_int=False,
    date_col='date',
    instrument_col='instrument'
)

m7 = M.join.v3(
    data1=m34.data,
    data2=m16.data,
    on='date,instrument',
    how='inner',
    sort=False
)

m53 = M.filter.v3(
    input_data=m7.data,
    expr='cond1&cond2&cond3',
    output_left_data=False
)

m13 = M.dropnan.v1(
    input_data=m53.data
)

m6 = M.stock_ranker_train.v5(
    training_ds=m13.data,
    features=m67.data_1,
    test_ds=m13.data,
    learning_algorithm='排序',
    number_of_leaves=20,
    minimum_docs_per_leaf=1000,
    number_of_trees=70,
    learning_rate=0.1,
    max_bins=1023,
    feature_fraction=1,
    m_lazy_run=False
)

m4 = M.instruments.v2(
    start_date='2018-01-01',
    end_date=T.live_run_param('trading_date', '2021-10-30'),
    market='CN_STOCK_A',
    instrument_list='000300.HIX',
    max_count=0
)

m35 = M.input_features.v1(
    features="""
# #号开始的表示注释，注释需单独一行
# 多个特征，每行一个，可以包含基础特征和衍生特征，特征须为本平台特征
close
instrument

"""
)

m36 = M.use_datasource.v1(
    instruments=m4.data,
    features=m35.data,
    datasource_id='bar1d_index_CN_STOCK_A',
    start_date='',
    end_date='',
    m_cached=False
)

m37 = M.instruments.v2(
    start_date='2018-01-01',
    end_date=T.live_run_param('trading_date', '2021-10-30'),
    market='CN_STOCK_A',
    instrument_list='',
    max_count=0
)

m38 = M.input_features.v1(
    features="""
# #号开始的表示注释，注释需单独一行
# 多个特征，每行一个，可以包含基础特征和衍生特征，特征须为本平台特征
bmret=close/shift(close,1)-1
"""
)

m39 = M.derived_feature_extractor.v3(
    input_data=m36.data,
    features=m38.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False,
    user_functions={}
)

m40 = M.select_columns.v3(
    input_ds=m39.data,
    columns_ds=m35.data,
    columns='',
    reverse_select=True
)

m43 = M.input_features.v1(
    features="""
# #号开始的表示注释，注释需单独一行
# 多个特征，每行一个，可以包含基础特征和衍生特征，特征须为本平台特征
close
open
low
high
adjust_factor"""
)

m42 = M.use_datasource.v1(
    instruments=m37.data,
    features=m43.data,
    datasource_id='bar1d_CN_STOCK_A',
    start_date='',
    end_date=''
)

m44 = M.input_features.v1(
    features="""
# #号开始的表示注释，注释需单独一行
# 多个特征，每行一个，可以包含基础特征和衍生特征，特征须为本平台特征
stockret=close/shift(close,1)-1"""
)

m45 = M.derived_feature_extractor.v3(
    input_data=m42.data,
    features=m44.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False,
    user_functions={}
)

m46 = M.select_columns.v3(
    input_ds=m45.data,
    columns_ds=m43.data,
    columns='',
    reverse_select=True
)

m41 = M.join.v3(
    data1=m40.data,
    data2=m46.data,
    on='date',
    how='inner',
    sort=False
)

m47 = M.input_features.v1(
    features="""
# #号开始的表示注释，注释需单独一行
# 多个特征，每行一个，可以包含基础特征和衍生特征，特征须为本平台特征
relative_ret=stockret-bmret
relative_ret_5=sum(relative_ret,5)
relative_ret_30=sum(relative_ret,30)"""
)

m48 = M.derived_feature_extractor.v3(
    input_data=m41.data,
    features=m47.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False,
    user_functions={}
)

m49 = M.filter.v3(
    input_data=m48.data,
    expr='(relative_ret_5>0)&(relative_ret_30>0)&(rank(relative_ret_30)>0.8)',
    output_left_data=False
)

m50 = M.select_columns.v3(
    input_ds=m49.data,
    columns='date,instrument',
    reverse_select=False
)

m51 = M.join.v3(
    data1=m18.data,
    data2=m50.data,
    on='date,instrument',
    how='inner',
    sort=False
)

m54 = M.filter.v3(
    input_data=m51.data,
    expr='cond1&cond2&cond3&cond4',
    output_left_data=False
)

m14 = M.dropnan.v1(
    input_data=m54.data
)

m8 = M.stock_ranker_predict.v5(
    model=m6.model,
    data=m14.data,
    m_lazy_run=False
)

m64 = M.select_columns.v3(
    input_ds=m14.data,
    columns='date,instrument,price_limit_status_0',
    reverse_select=False
)

m65 = M.join.v3(
    data1=m8.predictions,
    data2=m64.data,
    on='date,instrument',
    how='inner',
    sort=False
)

m55 = M.input_features.v1(
    features="""
# #号开始的表示注释，注释需单独一行
# 多个特征，每行一个，可以包含基础特征和衍生特征，特征须为本平台特征
#bm_0 = where(close/shift(close,5)-1<-0.05,1,0)

bm_0=where(ta_macd_dif(close,2,4,4)-ta_macd_dea(close,2,4,4)<0,1,0)"""
)

m56 = M.index_feature_extract.v3(
    input_1=m9.data,
    input_2=m55.data,
    before_days=100,
    index='000300.HIX'
)

m57 = M.select_columns.v3(
    input_ds=m56.data_1,
    columns='date,bm_0',
    reverse_select=False
)

m60 = M.input_features.v1(
    features="""
# #号开始的表示注释，注释需单独一行
# 多个特征，每行一个，可以包含基础特征和衍生特征，特征须为本平台特征
name"""
)

m59 = M.use_datasource.v1(
    instruments=m9.data,
    features=m60.data,
    datasource_id='instruments_CN_STOCK_A',
    start_date='',
    end_date=''
)

m58 = M.join.v3(
    data1=m59.data,
    data2=m57.data,
    on='date',
    how='left',
    sort=True
)

m61 = M.join.v3(
    data1=m65.data,
    data2=m58.data,
    on='date,instrument',
    how='left',
    sort=False
)

m62 = M.sort.v4(
    input_ds=m61.data,
    sort_by='score',
    group_by='date',
    keep_columns='--',
    ascending=False
)

m63 = M.trade.v4(
    instruments=m9.data,
    options_data=m62.sorted_data,
    start_date='',
    end_date='',
    initialize=m63_initialize_bigquant_run,
    handle_data=m63_handle_data_bigquant_run,
    prepare=m63_prepare_bigquant_run,
    before_trading_start=m63_before_trading_start_bigquant_run,
    volume_limit=0.025,
    order_price_field_buy='open',
    order_price_field_sell='close',
    capital_base=1000001,
    auto_cancel_non_tradable_orders=True,
    data_frequency='daily',
    price_type='后复权',
    product_type='股票',
    plot_charts=True,
    backtest_only=False,
    benchmark=''
)


[2021-12-03 13:47:57.779488] INFO: moduleinvoker: instruments.v2 开始运行..

[2021-12-03 13:47:57.848763] INFO: moduleinvoker: 命中缓存

[2021-12-03 13:47:57.851256] INFO: moduleinvoker: instruments.v2 运行完成[0.071815s].

[2021-12-03 13:47:57.866144] INFO: moduleinvoker: use_datasource.v1 开始运行..

[2021-12-03 13:47:57.878961] INFO: moduleinvoker: 命中缓存

...

[2021-12-03 13:56:15.923913] INFO: Performance: Simulated 928 trading days out of 928.

[2021-12-03 13:56:15.925850] INFO: Performance: first open: 2018-01-02 09:30:00+00:00

[2021-12-03 13:56:15.927524] INFO: Performance: last close: 2021-10-29 15:00:00+00:00

[2021-12-03 13:56:29.212635] INFO: moduleinvoker: backtest.v8 运行完成[181.492311s].

[2021-12-03 13:56:29.214873] INFO: moduleinvoker: trade.v4 运行完成[183.976761s].

In [2]:
#coding=utf-8
